In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
# Building experiments configs

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee

from copy import deepcopy

# TODO: move to one place with historical data experiments
fee_algos_to_consider = {
    "fixed_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "based_on_trade_count_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "adaptive_based_on_previous_block_price_move_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "discrete_fee_perfect_oracle": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
}

In [3]:
from experiments.experiment import SyntheticDataDescription, GBMParameters
from datetime import datetime
import numpy as np

# Provided data, see synthetic_data_example.ipynb

# volatile_market
# S0 = [3406.04, 1.299e-05]
# mu = [2.6092527015913387e-07, 0.0003608078612459716]
# cov_matrix = np.array([
#     [5.91363022e-07, 1.20049585e-06],
#     [1.20049585e-06, 3.40028539e-05]
# ])
# start_time = datetime(2024, 3, 1, 12, 0, 0)
# end_time = datetime(2024, 3, 2, 12, 0, 0)

# calm_market
S0 = [3179.96, 1.587e-05]
mu = [-5.439664535383297e-06, -3.1348041846428764e-06]
cov_matrix = np.array(
    [[1.62154349e-06, 1.31591401e-06],
    [1.31591401e-06, 1.92476086e-06]
])
start_time=datetime(2024, 8, 1, 12, 0, 0)
end_time=datetime(2024, 8, 31, 12, 0, 0)

# bull_market
# S0=[2518.75, 1.751e-05]
# mu=[8.82932404942772e-06, 1.0120694501213374e-05]
# cov_matrix = np.array(
#     [[1.04648385e-06, 1.37680192e-06],
#     [1.37680192e-06, 4.61054708e-06]
# ])
# start_time=datetime(2024, 11, 1, 12, 0, 0)
# end_time=datetime(2024, 11, 30, 12, 0, 0)

# bear_market
# S0=[3539.93, 2.858e-05]
# mu=[-3.5843786641630628e-06, -4.891135343153541e-06]
# cov_matrix = np.array(
#     [[9.69680101e-07, 1.32425664e-06],
#     [1.32425664e-06, 3.19313094e-06]
# ])
# start_time=datetime(2024, 4, 1, 12, 0, 0)
# end_time=datetime(2024, 4, 30, 12, 0, 0)

gbm_parameters = GBMParameters(S0=S0, mu=mu, cov_matrix=cov_matrix)

synthetic_data_description = SyntheticDataDescription(
    start_time=start_time,
    end_time=end_time,
    gbm_parameters=gbm_parameters,
)

random_seeds = [i for i in range(100)]

In [4]:
from experiments.experiment import Experiment
from experiments.configs import DEFAULT_UNINFORMED_USERS_CONFIG
from user.informed_user import InformedUser

# build experiments config

config_by_fee_algo = {}

for fee_algo_name, fee_algo in fee_algos_to_consider.items():
    config_by_fee_algo[fee_algo_name] = Experiment(
        data=synthetic_data_description, 
        fee_algorithm=deepcopy(fee_algo),
        informed_user=InformedUser(),
        uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
    )

In [5]:
from experiments.run_experiment import run_seeded_experiment

/Users/ir.v.lebedeva/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
from tqdm import tqdm

results_by_fee_algo = {}

for fee_algo_name, experiment in tqdm(config_by_fee_algo.items()):
    results_by_fee_algo[fee_algo_name] = run_seeded_experiment(experiment, random_seeds=random_seeds)

 25%|██▌       | 1/4 [28:24<1:25:12, 1704.23s/it]


KeyboardInterrupt: 

In [ ]:
all_experiments_results = {
    f"{fee_algo_name}_{random_seeds[i]}": results[i]
    for fee_algo_name, results in results_by_fee_algo.items()
    for i in range(len(results))
}

In [ ]:
from visualizations.compare_fee_algoritms import get_experiment_summary

df = get_experiment_summary(
    all_experiments_results
)

df['base_experiment_name'] = df['experiment_name'].str.extract(r'^(.*?)_\d+$')

In [9]:
df

,experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss,base_experiment_name
0,fixed_fee_0,72088.04,706,-7605.94,743,-71727.10,86871.67,fixed_fee
1,fixed_fee_1,97402.37,759,-8678.48,708,-96058.89,2249964.03,fixed_fee
2,fixed_fee_2,88193.43,714,-9461.41,761,-86107.02,3621769.83,fixed_fee
3,fixed_fee_3,70844.16,721,-8113.68,739,-70030.48,-71726.87,fixed_fee
4,fixed_fee_4,80918.41,741,-9442.17,724,-78801.24,1413659.80,fixed_fee
...,...,...,...,...,...,...,...,...
3995,discrete_fee_perfect_oracle_995,86895.75,627,-9159.79,706,-84400.97,5494646.32,discrete_fee_perfect_oracle
3996,discrete_fee_perfect_oracle_996,67996.15,609,-8783.27,731,-65912.88,461990.29,discrete_fee_perfect_oracle
3997,discrete_fee_perfect_oracle_997,73010.14,604,-9825.37,757,-69989.77,2553708.77,discrete_fee_perfect_oracle
3998,discrete_fee_perfect_oracle_998,57151.84,576,-8374.13,729,-55302.71,261612.04,discrete_fee_perfect_oracle


In [ ]:
# Volatile Market

import pandas as pd

grouped_stats = df.drop(columns=["experiment_name"]).groupby('base_experiment_name').agg(['mean', 'std'])

formatted_stats = grouped_stats.apply(
    lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
    axis=1
)

formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

formatted_stats_df.reset_index(inplace=True)

formatted_stats_df

,base_experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,adaptive_based_on_previous_block_price_move_fee,84742.41 ± 8409.84,730.88 ± 20.52,-8773.44 ± 587.48,719.42 ± 19.26,-83220.50 ± 8167.07,1274036.87 ± 1222603.55
1,based_on_trade_count_fee,86437.04 ± 8521.19,735.90 ± 20.36,-8783.72 ± 597.87,719.04 ± 19.35,-84928.07 ± 8271.01,1276421.91 ± 1223344.12
2,discrete_fee_perfect_oracle,69766.64 ± 7414.82,596.71 ± 20.93,-8804.84 ± 563.87,719.27 ± 18.86,-67541.67 ± 7193.84,1256551.09 ± 1221397.47
3,fixed_fee,85640.18 ± 8477.36,733.52 ± 20.56,-8768.77 ± 611.06,718.93 ± 19.32,-84133.68 ± 8222.04,1275484.21 ± 1223233.44


In [ ]:
# Calm Market

import pandas as pd

grouped_stats = df.drop(columns=["experiment_name"]).groupby('base_experiment_name').agg(['mean', 'std'])

formatted_stats = grouped_stats.apply(
    lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
    axis=1
)

formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

formatted_stats_df.reset_index(inplace=True)

formatted_stats_df

In [ ]:
# Bull Market

import pandas as pd

grouped_stats = df.drop(columns=["experiment_name"]).groupby('base_experiment_name').agg(['mean', 'std'])

formatted_stats = grouped_stats.apply(
    lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
    axis=1
)

formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

formatted_stats_df.reset_index(inplace=True)

formatted_stats_df

In [ ]:
# Bear Market

import pandas as pd

grouped_stats = df.drop(columns=["experiment_name"]).groupby('base_experiment_name').agg(['mean', 'std'])

formatted_stats = grouped_stats.apply(
    lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
    axis=1
)

formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

formatted_stats_df.reset_index(inplace=True)

formatted_stats_df